# Ενδίαμεση Εργασία

### Θεολόγης Γεώργιος , ΑΕΜ:10413 

### email: gtheolog@ece.auth.gr

### Τμήμα Ηλεκτρολόγων Μηχανικών και Μηχανικών Υπολογιστών

Αντικείμενο της παρούσας εργασίας είναι η ανάπτυξη αλγορίθμων ταξινόμησης και η σύγκριση της απόδοσης τους πάνω σε ένα dataset δεδομένων.

Συγκεκριμένα υλοποιήθηκαν :

* Kατηγοριοποιητής πλησιέστερου γείτονα με 1 πλησιέστερο γείτονα (Nearest Neighbor k=1)
* Kατηγοριοποιητής πλησιέστερου γείτονα με 3 πλησιέστερους γείτονες (Nearest Neighbor k=3)
* Κατηγοριοποιητή πλησιέστερου κέντρου κλάσης (Nearest Class Centroid)

Ο κώδικας υλοποιήθηκε έχοντας ως βάση τα tensor της βιβλιοθήκης torch. Με την χρήση των tensor μπορούμε να πραγματοποίησουμε της απαιτούμενες πράξεις στην GPU του υπολογιστή μας και συνεπώς να μειώσουμε τον χρόνο εκτέλλεσης παραλληλοποιόντας την διαδικασία.

Ως μέτρο σύγκρισης της ορθότητας των αλγορίθμων που αναπτύχθηκαν χρησιμοποιήθηκαν οι έτοιμοι αλγόριθμοι της sklearn. 

Η εφαρμογή των αλγορίθμων έγινε σε 2 βάσεις δεδομένων:

* CIFAR-10
* Stellar Classification Dataset - SDSS17


# Βιβλιοθήκες και Εργαλεία

In [65]:
#Χρήσιμες βιβλιοθηκές και εργαλεία
import numpy as np    #Για υπολογισμούς και συμβατότητα με την sklearn
import matplotlib.pyplot as plt 
import torch  #χρήση Tensors και δυνατότητα πράξεων με Tensors στην GPU
import pandas as pd #Χρηση Dataframes
from sklearn.model_selection import train_test_split #Για εύκολη κατάτμηση του Dataset
from sklearn.preprocessing import LabelEncoder    #Για γρήγορη αντιστοιχίση string labels σε αριθμητικά
from sklearn.neighbors import KNeighborsClassifier  #Αλγόριθμος k-ΝΝ της sklearn
from sklearn.neighbors import NearestCentroid  #Αλγόριθμος Nearest Centroid της sklearn
from sklearn.metrics import accuracy_score    #Αλγόριθμος για ποσοστό επιτυχίας της ταξινόμησης των testing data απο τον ταξινομητή της sklearn
from sklearn.decomposition import PCA   #Principal Component Analysis της sklearn
from sklearn.preprocessing import StandardScaler #Για γρήγορο scaling των δεδομένων της βάσης βοηθάει στην καλλήτερη εφαρμογή του PCA

#Ελέγχουμε ότι μπορούμε να χρησιμοποιήσουμε την GPU 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")




Using device: cuda


# Κώδικας Ταξινομητών

### Συνάρτηση για υπολογισμό ευκλείδιας απόστασης

In [66]:
def distance(x1,x2):
    dist = torch.sqrt(torch.sum(torch.square(x1 - x2), dim=1)) 
    return dist
    

### Κλάση για υλοποιήση της λειτουργίας του ταξινομητή πλησιέστερου γείτονα για k πλησιέστερους γείτονες

In [67]:
class k_NN_class:
    def __init__(self,k,dev):
        self.k=k
        self.label_indent=[]
        self.device=dev

    def train(self,xtrain,ltrain):
        self.xtrain=xtrain.to(self.device)
        self.ltrain=ltrain.to(self.device)

    def identify(self,xtest):
        for i in range(len(xtest)):
            dis_tensor=distance(xtest[i],self.xtrain)
            sorted_indexes=torch.argsort(dis_tensor)
            minimum_label_indexes=self.ltrain[sorted_indexes[:self.k]]
            unique_elements, counts = torch.unique(minimum_label_indexes, return_counts=True)
            index_of_label= unique_elements[torch.argmax(counts).item()].item()
            self.label_indent.append(index_of_label)
        return self.label_indent  
    
    def accuracy(self,label_test):
        arr_of_error=torch.tensor(self.label_indent).to(self.device)-label_test
        num_zeros = torch.count_nonzero(arr_of_error==0)
        accuracy=num_zeros/len(label_test)*100
        return accuracy


Αρχικά απαιτείται η αρχικοποιήση της παραμέτρου του πλήθους πλησιέστερων γειτόνων k που λαμβάνονται υπόψιν από τον αλγόριθμο καθώς και η συσκεύη που θα πραγματοποιήθουν οι πράξεις μεταξύ των δεδομένων.

Επείτα πραγματοποιείται η εκπαίδευση με την χρήση της  **self.train(self,xtrain,ltrain)** η οποία για αυτόν τον ταξινομήτη απλά αποθηκεύει τα δεδομένα εκπαίδευσης.

Μετά την εκπαίδευση, γίνεται η ταξινόμηση των testing data με την χρήση της  **self.identify(self,xtest)**. Πρακτικά για κάθε στοιχείο των δεδομένων ελέγχου υπολογίζεται η ευκλείδια απόσταση του από κάθε δεδομένου εκπαίδευσης. Ύστερα ταξινομούμε τις αποστάσεις ώστε να βρούμε τα k (1 η 3) index (σείρα στον tensor) των δεδομένων εκπαίδευσης που απέχουν την ελάχιστη απόσταση από το συγκεκρίμενο δεδομένου ελέγχου. Επείτα διαλέγουμε το πιο σύχνο label από τα k (αν k=1 θα είναι το ελάχιστο) και ταξινομούμε το συγκεκριμένο δεδομένο εκπαίδευσης στην κλάση που αντιστοιχεί το label. Έτσι αντιστοιχείται κάθε δεδομένου ελέγχου σε κάποια κλάση ενώ δημιουργείται η λίστα label_indent που θα συγκρίθει με τα πραγματικά label των δεδομένων εκπαίδευσης για να δούμε τι πόσοστο επιτυχίας είχαμε στην ταξινόμηση.

Έτσι, το πόσοστο της επιτυχίας προσδιορίζεται από την **self.accuracy(self,label_test)** 


In [68]:
class centroid_NN_class:
    def __init__(self,dev):
        self.label_indent=[]
        self.device=dev
        self.data_centroid=[]
        self.label_centroid=[]


    def train(self,xtrain,ltrain):
        unique_value = torch.unique(ltrain)
        for value in unique_value :
            indexes_of_value=torch.where(ltrain == value)[0]
            mean_of_class = xtrain[indexes_of_value].mean(dim=0)
            self.data_centroid.append(mean_of_class.tolist())
            self.label_centroid.append(value)
        self.data_centroid=torch.tensor(self.data_centroid).to(self.device)
        self.label_centroid=torch.tensor(self.label_centroid).to(self.device)

    def identify(self,xtest):
        for i in range(len(xtest)):
            dis_tensor=distance(xtest[i],self.data_centroid)
            sorted_indexes=torch.argsort(dis_tensor)
            index_of_min=(torch.argmin(dis_tensor)).item()
            tes_i=self.label_centroid[index_of_min]
            self.label_indent.append(tes_i.item())
        return self.label_indent  
    
    def accuracy(self,label_test):
        arr_of_error=torch.tensor(self.label_indent).to(self.device)-label_test
        num_zeros = torch.count_nonzero(arr_of_error==0)
        accuracy=num_zeros/len(label_test)*100
        return accuracy

Η λογική στον ταξινόμητη πλησίεστερου κέντρου κλάσης είναι παρόμοια με αυτήν του k πλησιέστερου γείτονα. 
Απλά κατά την διαδικασία εκπαίδευσης υπολογίζεται η μέση τιμή ολών των features των δεδομένων εκπαίδευσης ανά κλάση. Έτσι παίρνουμε ένα διάνυσμα γνωρισμάτων για κάθε κλάση (το κέντρο της κλάσης). Στην διαδικασία ταξινόμησης (identify-predict) το δεδομένο ελέγχου αντιστοιχίζεται στο πλησιέστερο κέντρο κλάσης.

Στη συνέχεια υπολογίζεται ξανά το ποσοστό επιτυχίας στην ταξινόμηση.

Η συνάρτηση **my_classification_algorithms** που ορίζεται παρακάτω χρησιμοποιείται για να εκτελέσει και τους 3 αλγόριθμους ταξινόμησης που υλοποιήθηκαν παραπάνω και για να μας δώσει το πόσοστο ακρίβειας τους ως προς την ταξινόμηση.

In [69]:
def my_classification_algorithms(device,data_train,data_test,label_train,label_test):
    #Ταξινομητής πλησιέστερου γείτονα με k=1 άμεσα εφαρμοσμένο στις εικόνες
    torch.cuda.empty_cache() #empty cache
    k_1_NN=k_NN_class(1,device)
    k_1_NN.train(data_train,label_train)
    label_i=k_1_NN.identify(data_test.to(device))
    accuracy=k_1_NN.accuracy(label_test.to(device)).item()
    print(f"Accuracy of Nearest-Neighbour k=1 classification: {accuracy:.2f}%")

    #Ταξινομητής πλησιέστερου γείτονα με k=3 άμεσα εφαρμοσμένο στις εικόνες
    torch.cuda.empty_cache()
    k_3_NN=k_NN_class(3,device)
    k_3_NN.train(data_train,label_train)
    label_i=k_3_NN.identify(data_test.to(device))
    accuracy=k_3_NN.accuracy(label_test.to(device)).item()
    print(f"Accuracy of Nearest Neighbour k=3 classification: {accuracy:.2f}%")

    #Ταξινομητής πλησιέστερου κένρου άμεσα εφαρμοσμένο στις εικόνες
    torch.cuda.empty_cache()
    nearest_centroid=centroid_NN_class("cuda") 
    nearest_centroid.train(data_train,label_train)
    label_i=nearest_centroid.identify(data_test.to(device))
    accuracy=nearest_centroid.accuracy(label_test.to(device)).item()
    print(f"Accuracy of Nearest Centroid classification: {accuracy:.2f}%")

Αντίστοιχα η συνάρτηση που κάνει το ίδιο με τους έτοιμους αλγορίθμους της sklearn:

In [70]:
def sklearn_classifiers(data_train,data_test,label_train,label_test):
    #####K-Nearest Neighbors classifier with k=1
    knn_1 = KNeighborsClassifier(n_neighbors=1,metric='euclidean')

    # Train with the training data
    knn_1.fit(data_train, label_train)

    # Predict test set label
    label_pred= knn_1.predict(data_test)

    # Calculate the accuracy of classification
    accuracy = accuracy_score(label_test, label_pred)
    print(f"Accuracy of Nearest-Neighbour k=1 classification:  {accuracy * 100:.2f}%")

    ######K-Nearest Neighbors classifier with k=3
    knn_3 = KNeighborsClassifier(n_neighbors=3,metric='euclidean')

    # Train with the training data
    knn_3.fit(data_train, label_train)

    # Predict test set label
    label_pred= knn_3.predict(data_test)

    # Calculate the accuracy of classification
    accuracy = accuracy_score(label_test, label_pred)
    print(f"Accuracy of Nearest-Neighbour k=3 classification:  {accuracy * 100:.2f}%")

    #####Nearest Centroid
    clf = NearestCentroid()
    # Train with the training data
    clf.fit(data_train, label_train)
    # Predict on the test set
    label_pred = clf.predict(data_test)

    # Evaluate accuracy
    accuracy = accuracy_score(label_test, label_pred)
    print(f"Accuracy of Nearest Centorid classification:  {accuracy * 100:.2f}%")




## Εισαγωγή της βάσης δεδομένων CIFAR-10

Αρχικά θα ελέγξουμε τους αλγορίθμους που αναπτύχθηκαν στην βάση δεδομένων cifar-10. Μπορούμε να την βρούμε στην σελίδα: https://www.cs.toronto.edu/~kriz/cifar.html

Αυτή η βάση δεδομένων περιέχει εικόνες που αντιστοιχούν σε 10 κλάσεις οντοτήτων (airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
Τα labels των δεδομένων έχουν αντιστοιχίσει την κάθε τάξη σε ένα νούμερο από 0-9.
Τα data δεδομένα περιέχουν 60000 εικόνες , 6000 για κάθε κλάση. Κάθε εικόνα έχει μέγεθος 3072 (32*32=1024 pixels και 3 color channels).

Ακολουθεί η εισαγωή των δεδομένων των εικονών (data) και των ετικετών τους (labels):

In [71]:
def unpickle(file): #ανάγνωση των αρχείων
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [72]:

dt1=unpickle("C:\\CIFAR\\cifar-10-batches-py\\data_batch_1")
dt2=unpickle("C:\\CIFAR\\cifar-10-batches-py\\data_batch_2")
dt3=unpickle("C:\\CIFAR\\cifar-10-batches-py\\data_batch_3")
dt4=unpickle("C:\\CIFAR\\cifar-10-batches-py\\data_batch_4")
dt5=unpickle("C:\\CIFAR\\cifar-10-batches-py\\data_batch_5")
dtest=unpickle("C:\\CIFAR\\cifar-10-batches-py\\test_batch")

In [73]:
#εισαγωγή των δεδομένων σε μορφή tensor. Εύκολα μπορούν να μετασχηματιστούν στην συνέχεια σε Numpy arrays.
data_train_cifar10= torch.cat((torch.tensor(dt1[b'data'], dtype=torch.float32),torch.tensor(dt2[b'data'], dtype=torch.float32),torch.tensor(dt3[b'data'], dtype=torch.float32),torch.tensor(dt4[b'data'], dtype=torch.float32),torch.tensor(dt5[b'data'], dtype=torch.float32)), dim=0)
label_train_cifar10=torch.cat((torch.tensor(dt1[b'labels'], dtype=torch.float32),torch.tensor(dt2[b'labels'], dtype=torch.float32),torch.tensor(dt3[b'labels'], dtype=torch.float32),torch.tensor(dt4[b'labels'], dtype=torch.float32),torch.tensor(dt5[b'labels'], dtype=torch.float32)), dim=0)

data_test_cifar10=torch.tensor(dtest[b'data'], dtype=torch.float32)
label_test_cifar10=torch.tensor(dtest[b'labels'], dtype=torch.float32)


Τα training data αποτελούνται απο 50000 εικόνες. 

Τα testing data αποτελούνται απο 10000 εικόνες. 

In [94]:
#Εισαγωγή των εικόνων και των ετικετών στα placeholder ονόματα που θα χρησιμοποιήθουν για λόγους αντιστοιχίας και στην υπόλοιπη εργασία:
data_train=data_train_cifar10
label_train=label_train_cifar10
data_test=data_test_cifar10
label_test=label_test_cifar10

## Απόδοση των ταξινόμητων σε άμεση εφαρμογή στις εικόνες

Αρχικά θα δούμε την απόδοση των ταξινόμητων σε άμεση εφαρμογή πάνω στις εικόνες της βάσης δεδομένων. Δηλαδή ώς στοιχείο της εκπαίδευσης και ελέγχου είναι ένα διάνυσμα **3072** τιμών-γνωρισμάτων.

Τα πρώτα 32*32=1024 γνωρίσματα αναφέρονται στο red του RGB κώδικα, τα επόμενα 1024 στο green, και τα τελευταία 1024 στο blue.

In [75]:
print("Classification algorithms that were implemented with all features:")
my_classification_algorithms(device,data_train,data_test,label_train,label_test)

Classification algorithms that were implemented with all features:
Accuracy of Nearest-Neighbour k=1 classification: 35.39%
Accuracy of Nearest Neighbour k=3 classification: 33.03%
Accuracy of Nearest Centroid classification: 27.74%


Από ότι βλέπουμε η απόδοση των αλγορίθμων ταξινόμησης είναι σχετικά μικρή καθώς αυτή η άμεση εφαρμογή τους μας δίνει για κάθε αλγόριθμο τις εξής επιτυχίες ταξινόμισης:

**1-nn**:35.39 %

 **3-nn**:33.03 % 
 
 **centroid**: 27.74%  


Όλες αυτές είναι κάτω του 40 %. Τα αποτελέσμα αυτό είναι διαισθητικά λογικό καθώς κάθενα γνώρισμα από τα 3072 χρησιμοποιούμενα δεν είναι παρά μία ένδειξη χρώματος RGB για ένα pixel μόνο της εικόνας. Αυτή η αναπαράσταση λοιπόν των γνωρισμάτων δεν μπορεί να φέρει κάποια άμεση πληροφορία σχετικά με την κλάση που αντιστοιχεί στην εικόνα και επομένως τα ποσοστά επιτυχούς ταξινόμησης παραμένουν αρκετά χαμηλά. 

Μεταξύ των 3 αλγορίθμων , οι αλγόριθμοι του πλησιέστερου γείτονα παρουσίαζουν καλύτερη απόδοση απο τον πλησιέστερου κέντρου κλάσης. Αυτό είναι αρκετά λογικό καθώς στον πλησιέστερο γείτονα μία εικόνα ελέγχου μπορεί να συγκριθεί με εικόνες που ως προς τα RGB γνωρίσματα ανα pixel να είναι ποίο κοντά σε κάποια άλλη εικόνα της ίδιας κλάσης του συνόλου εκπαίδευσης από οτί θα ήταν στον μέσο όρο όλων των εικονών της κλάσης.  

Θα συνεχίσουμε με την προ-επεξεργασία της εικόνας με σκοπό να βγάλουμε καλύτερα αποτελέσματα. 

Παρατείθεται και η απόδοση που δίνουν οι αλγόριθμοι της sklearn:

In [76]:
print("Classification algorithms of sklearn:")
sklearn_classifiers(data_train.numpy(),data_test.numpy(),label_train.numpy(),label_test.numpy())

Classification algorithms of sklearn:
Accuracy of Nearest-Neighbour k=1 classification:  35.39%
Accuracy of Nearest-Neighbour k=3 classification:  33.03%
Accuracy of Nearest Centorid classification:  27.74%


## Χρήση PCA (Principal Component Analysis) 


Με την Ανάλυση Κύριων Συνιστωσών (PCA) μπορούμε να προ-επεξεργαστούμε τις εικόνες κατάλληλα ώστε να μειώσουμε την διάσταση των γνωρισμάτων των συνόλων εκπαίδευσης και ελέγχου. Η μείωση αυτή της διάστασης των γνωρισμάτων των δεδομένων τα βάση των οποίων θα ταξινομήσουμε τις εικόνες μας συμφέρει αρκετά καθώς μειώνεται το πλήθος των υπολογισμών και η πολυπλοκότητα στην ταξινόμηση. Επίσης κατά αυτήν την διαδικασία κάθε γνώρισμα που προκύπτει τείνει να είναι γραμμικά ασυσχέτιστο από τα άλλα και να συγκεντρώνει μεγαλύτερη ποσότητα χρήσιμης πληροφορίας δημιουργώντας ένα πιο ουσιαστικό feature space.

Θα χρησιμοποιήσουμε τις έτοιμες κλάσεις StandardScaler και PCA για να κανονικοποίση των δεδομένων (μέση τιμή=0 και τυπική απόκλιση=1) και ανάλυση κυριών συνιστωσών αντίστοιχα.

In [95]:
scaler = StandardScaler()
pca = PCA(n_components=50)  # Reduce to 50 principal components

data_train_np=data_train.numpy().astype(np.float32)
data_train_scaled = scaler.fit_transform(data_train_np)
data_train_pca = pca.fit_transform(data_train_scaled)

data_test_np=data_test.numpy().astype(np.float32)
data_test_scaled = scaler.transform(data_test_np)
data_test_pca = pca.transform(data_test_scaled)

data_train_pca=torch.from_numpy(data_train_pca).to(torch.float32)
data_test_pca=torch.from_numpy(data_test_pca).to(torch.float32)


Ακολουθεί η εφαρμογή των αλγορίθμων ταξινόμησης πάνω στα data:

In [96]:
print("Classification algorithms that were implemented with all features:")
my_classification_algorithms(device,data_train_pca,data_test_pca,label_train,label_test)

Classification algorithms that were implemented with all features:
Accuracy of Nearest-Neighbour k=1 classification: 38.94%
Accuracy of Nearest Neighbour k=3 classification: 38.11%
Accuracy of Nearest Centroid classification: 27.95%


Παρατηρούμε ότι είχαμε μιά μικρή αύξηση του ποσοστού επιτυχίας και για τους τρείς αλγορίθμους. Ώστοσο, καθώς το ποσοστό επιτυχίας και για τα τρία παραμένει κάτω απο 40 % βλέπουμε ότι οι κύριες συνιστώσες που δημιουρήθηκαν δεν έφεραν πολύ πιο σημαντική και διακριτή πληροφορία ως προς τις κλάσεις σε σχέση με πριν την εφαρμογή του pca.

Για την πιο αποτελεσματική ταξινόμηση των εικονών καλό είναι τα γνωρίσματα να φέρουν πληροφορία άμεσα σχετιζόμενη με το αντικείμενο της κλάσης και όχι απλά πληροφορία RGB ανά pixel. Για παράδειγμα ένας edge detector που θα διακρίνει τις ακμές του αντικειμένου της εικόνας να δώσει καλύτερα features βάση των οποίων θα πραγματοποιηθεί η ταξινόμηση της εικόνας. 

Από τα παραπάνω μπορoύμε να συμπεράνουμε ότι οι αλγόριθμοι nearest neighbour και nearest centroid δεν εμφανίζουν μεγάλα ποσοστά επιτυχίας ως προς την διαδικασία ταξινόμισης εικονών σε συγκεκριμένο πλήθος τάξεων. Η απόδοση τους μπορεί να αυξηθεί μετά από την κατάλληλη προεπεξεργασία των εικονών (pca,edge_detection) και προχωρημένου feature engineering.

Η χρήση άλλων μοντέλων ταξινόμησης όπως ένα νευρωνικό δίκτυο ενδέχεται να αυξήσει το ποσοστό επιτυχίας της σωστής ταξινόμησης των δεδομένων της βάσης δεδομένων cifar-10.

Θα γίνει με την ανάπτυξη του νευρωνικού στη συνέχεια της εργασίας απόπειρα να προσθέσουμε και άλλες μεθόδους προ-επεξεργασίας της εικόνας με την ελπίδα να βελτιώσουμε τα αποτελέσματα της ταξινόμησης που θα πραγματοποιεί το μοντέλο που θα αναπτύξουμε.

Παρατείθεται και η απόδοση που δίνουν οι αλγόριθμοι της sklearn:

In [97]:
print("Classification algorithms of sklearn with pca=50 features:")
sklearn_classifiers(data_train_pca.numpy(),data_test_pca.numpy(),label_train.numpy(),label_test.numpy())

Classification algorithms of sklearn with pca=50 features:
Accuracy of Nearest-Neighbour k=1 classification:  38.94%
Accuracy of Nearest-Neighbour k=3 classification:  38.11%
Accuracy of Nearest Centorid classification:  27.95%


## Εισαγωγή της βάσης δεδομένων Stellar Classification Dataset - SDSS17



Η βάση αυτή περίεχει την συλλογή 100,000 παρατηρήσεων διαστημικών αντικειμένων με την χρήση του SDSS (Sloan Digital Sky Survey).

Το dataset αντλήθηκε από τον εξής σύνδεσμο: https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17

Περιλαμβάνει για κάθε αντικείμενο 17 features ενώ υπάρχουν **3 κλάσεις** στις οποίες μπορεί να αντισοιχισθεί το εκάστοτε αντικείμενο.
Συγκεκριμένα , οι 3 κλάσεις είναι:
* Γαλαξίας (GALAXY) 
* Αστέρι (STAR)
* Κβάζαρ (QSO)

In [80]:
data_star=pd.read_csv('C:\\read_data\\star_classification.csv')
data_star.head() # ενδεικτική παρουσίαση των features και της κλάσης των πρώτων 5 αντικειμένων

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


Από αυτά τα γνωρίσματα παρατηρούμε ότι αριθμοί ID που αφορούν το μοναδικό προσδιοριστικό του αντικειμένου και των εργαλείων παρατήρησης στην βάση δεδομένων δεν δίνουν καμία πληροφορία για την ίδια την κλάση (obj_ID,spec_obj_ID,fiber_ID). Καμια πληροφορία για την κλάση δεν δίνει ούτε η θέση στον ουράνιο θόλο (alpha,delta) , όυτε η ημερομηνία της παρατήρησης αλλά και ούτε και specification του εξοπλισμού του πειράματος.

Ουσιαστικό πληροφορία για τις κλάσεις πέρνουμε από τα γνωρίσματα που αφορούν μετρήσεις φωτεινότητας/ακτινοβολίας των αντικειμένων και της μετατόπιση προς το ερυθρό.
Άρα τα γνωρίσματα-features που θα χρησιμοποιήσουμε στην ταξινόμηση είναι τα εξής:
* u = Ultraviolet filter in the photometric system
* g = Green filter in the photometric system
* r = Red filter in the photometric system
* i = Near Infrared filter in the photometric system
* z = Infrared filter in the photometric system
* redshift = redshift value based on the increase in wavelength



In [81]:
print(data_star["class"].value_counts()) #print frequency of each class


class
GALAXY    59445
STAR      21594
QSO       18961
Name: count, dtype: int64


Άρα η πλειονότητα των στοιχείων του dataset (60%) αφορούν γαλαξίες και επείτα από 20 % πίανουν τα κβάζαρ και τα αστέρια. Συνεπώς τα δείγματα του στοιχεία του dataset δεν είναι ισομοιρασμένα όπως ήταν στο cifar-10. Ωστόσο ακόμα και στην φύση είναι δύσκολο να παρατηρήσουμε παντά ισομοιρασμό των πειραματικών δεδομένων σε ίσο αριθμό ανά κλάση. Για αυτό θα χρησιμοποιηθεί η 

In [82]:
sdss_data=data_star.loc[:, ["u", "g","r","i","z","redshift"]].to_numpy()
label_sdss =data_star["class"].to_numpy()
print(label_sdss )
map_2_num = {"GALAXY": 0, "STAR": 1, "QSO": 2}  #κανω αντιστοιχία των string ονομάτων της κάθε κλάσης σε έναν αριθμό.
label_sdss=np.array([map_2_num[string] for string in label_sdss]) 
print(label_sdss)

['GALAXY' 'GALAXY' 'GALAXY' ... 'GALAXY' 'GALAXY' 'GALAXY']
[0 0 0 ... 0 0 0]


Στη συνέχεια θα μοιράσουμε τα δεδομένα σε σύνολο εκπαίδευσης (data_train,label_train) και σε σύνολο ελέγχου (data_test,label_test). Το σύνολο εκπαίδευσης θα έχει 60 % των δεδομένων ενώ το σύνολο ελέγχου το 40%. Επίσης , επιλέγουμε για αρχή το seed για τον τρόπο που γίνεται ο διαμοιρασμός ίσο με 28. Έγινε δοκιμή πολλών seed και τα παρακάτω αποτελέσματα ως προς την αποδοτικότητα των ταξινομητών δεν διέφεραν σημαντικά.

In [83]:
data_train, data_test, label_train, label_test = train_test_split(sdss_data, 
                                                    label_sdss, 
                                                    test_size=0.4, # 40% test, 60% train
                                                    random_state=28) 
print(f"size of training_data: {len(data_train)}")
print(f"size of testing_data: {len(data_test)}")
data_train=torch.tensor(data_train)
data_test=torch.tensor(data_test)
label_train=torch.tensor(label_train)
label_test=torch.tensor(label_test)


size of training_data: 60000
size of testing_data: 40000


In [84]:
print("Classification algorithms that were implemented:")
my_classification_algorithms(device,data_train,data_test,label_train,label_test)

Classification algorithms that were implemented:
Accuracy of Nearest-Neighbour k=1 classification: 94.07%
Accuracy of Nearest Neighbour k=3 classification: 94.94%
Accuracy of Nearest Centroid classification: 56.69%


Βλέπουμε ότι τα ποσοστά επιτυχίας της ταξινόμησης με χρήση του Nearest Neighbour για k=1 και k=3 (άνω του 90 %) και ενώ ο Nearest Centroid εξακολουθεί να υπολείπεται αυτών τώρα ξεπερνάει το 50 % επιτυχούς ταξινόμησης.

Η αυξήση του ποσοστού επιτυχίας των αλγορίθμων από το cifar-10 στο sdss2017 οφείλεται στο οτι το sdss database πρώτον έχει μικρότερο αριθμό κλάσεων στους οποίους πραγματοποιείται η ταξινόμηση (3 αντί για 10 προηγουμένως) καθώς και ότι τα features είναι πολύ λιγότερα και φέρουν σημαντικότερη ποσότητα πληροφορίας για την κλάση από οτι φέρανε τα 3072 γνωρίσματα της αρχικής εικόνας ή οι 50 κύριες συνιστώσες που προέκυψαν μετά το pca. Επίσης το γεγονός ότι τα data εδώ ήταν στην πλειονότητα γαλαξίες 60 % ίσως προμηνύει ότι ο ταξινόμητης ευνοιεί τα testing data που περιέχουν μεγάλο πλήθος  



Παρατείθεται και η απόδοση που δίνουν οι αλγόριθμοι της sklearn:

In [85]:
print("Classification algorithms of sklearn:")
sklearn_classifiers(data_train.numpy(),data_test.numpy(),label_train.numpy(),label_test.numpy())

Classification algorithms of sklearn:
Accuracy of Nearest-Neighbour k=1 classification:  94.07%
Accuracy of Nearest-Neighbour k=3 classification:  94.94%
Accuracy of Nearest Centorid classification:  56.69%


Θα δοκιμάσουμε να αλλάξουμε τα σύνολα διαίρεσης του συνολικού dataset σε 30% test και 70 % training καθώς και θα βάλουμε το seed=10

In [86]:
data_train, data_test, label_train, label_test = train_test_split(sdss_data, 
                                                    label_sdss, 
                                                    test_size=0.3, # 40% test, 60% train
                                                    random_state=10) 

print(f"size of training_data: {len(data_train)}")
print(f"size of testing_data: {len(data_test)}")
data_train=torch.tensor(data_train)
data_test=torch.tensor(data_test)
label_train=torch.tensor(label_train)
label_test=torch.tensor(label_test)

size of training_data: 70000
size of testing_data: 30000


In [87]:
print("Classification algorithms that were implemented:")
my_classification_algorithms(device,data_train,data_test,label_train,label_test)

Classification algorithms that were implemented:
Accuracy of Nearest-Neighbour k=1 classification: 94.17%
Accuracy of Nearest Neighbour k=3 classification: 95.06%
Accuracy of Nearest Centroid classification: 56.65%


Τα ποσοστά της επιτυχίας είναι αντίστοιχα με τα προηγούμενα. Όσο και να αλλάξουμε το ποσοστό του test set στο 20-40 % των συνολικών δεδομένων καθώς και τo seed με το οποίο γίνεται ο διαμοιρασμός των δεδομένων στα δύο σύνολα τα ποσοστά παραμένουν σχετικά αμετάβλητα.

Συνεπώς, το συγκεκριμένο dataset εμφανίζει υψηλό ποσοστό επιτυχίας ως προς την ταξινόμηση με την εφαρμογή των αλγορίθμων πλησιέστερου γείτονα (με 1 και 3 πλησιέστερους γείτονες) ενώ η απόδοση του πλησιέστερου κέντρου εξακολουθεί να είναι σημαντικά μικρότερη από του πλησιέστερου γείτονα ακριβώς γιατί ο αλγόριθμος πλησιέστερου γείτονα μπορεί να βρεί αντιστοιχίσει ένα δεδομένο ελέγχου μιας κλάσης σε πολύ κοντινά δεδομένα εκπαίδευσης που αντιστοιχούν στην ίδια κλάση ενώ ο πλησιέστερου κέντρου λόγω πιθανής διαφοράς στην διασπορά των δεδομένων μπορεί να οδηγήσει δεδομένα μιας κλάσης να είναι πιο κοντά στο κέντρο μιας άλλης κλάσης από ότι στο δικό τους κέντρο.

Τελός θα επιχειρήσουμε να κανονικοποιήσουμε και τα δεδομένα δηλαδή να αφαιρέσουμε την μέση τιμή τους να τα διαρέσουμε με την τυπική απόκλιση τους ώστε να δούμε άμα αυτο βοηθάει την διαδικασία της ταξινόμησης.



In [90]:
data_train, data_test, label_train, label_test = train_test_split(sdss_data, 
                                                    label_sdss, 
                                                    test_size=0.4, # 40% test, 60% train
                                                    random_state=28) 

mean_train = np.mean(data_train, axis=0)
std_dev_train = np.std(data_train, axis=0)

# Normalize data both train and test set by the statistics of the training set
data_train = (data_train - mean_train) / std_dev_train
data_test=(data_test - mean_train) / std_dev_train
data_train=torch.tensor(data_train)
data_test=torch.tensor(data_test)
label_train=torch.tensor(label_train)
label_test=torch.tensor(label_test)

print("Classification algorithms that were implemented:")
my_classification_algorithms(device,data_train,data_test,label_train,label_test)

Classification algorithms that were implemented:
Accuracy of Nearest-Neighbour k=1 classification: 95.48%
Accuracy of Nearest Neighbour k=3 classification: 96.33%
Accuracy of Nearest Centroid classification: 60.68%


Τα ποσοστά επιτυχίας πλησιέστερου γείτονα είναι παρόμοια με πρίν ενώ το ποσοστό επιτυχίας πλησιέστερου κέντρου αυξήθηκε λίγο ενδεχομένως από την κανονικοποιήση των δεδομένων.